In [113]:
import torch.nn as nn
import torch
import torch.backends.cudnn as cudnn
import torch.optim as optim
import numpy as np
import scipy
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold as skf

In [114]:
import os
import torch.backends.cudnn as cudnn
import torch.utils.data


def test(dataset_loader,epoch,model,name):
    cuda = True
    cudnn.benchmark = True
    batch_size = 100
    image_size = 32
    alpha = 0

    dataloader = dataset_loader

    """ training """

    my_net = model
    my_net = my_net.eval()

    if cuda:
        my_net = my_net.cuda()

    len_dataloader = len(dataloader)
    data_target_iter = iter(dataloader)

    i = 0
    n_total = 0
    n_correct = 0

    while i < len_dataloader:

        # test model using target data
        data_target = next(data_target_iter)
        test_img, test_label = data_target

        batch_size = len(test_label)

        if cuda:
            test_img = test_img.cuda()
            test_label = test_label.cuda()

        class_output, _ = my_net(input_data=test_img.float(), alpha=alpha)
        _,pred = torch.max(class_output, dim=1)
        _,gt = torch.max(test_label,dim=1)
        n_correct += (pred == gt).sum().item()
        n_total += batch_size

        i += 1

    accu = 100*n_correct/ n_total
    print(f'Epoch {epoch} for {name} dataset is {accu}')
    return accu


In [115]:
import torch.nn as nn
from torch.autograd import Function


class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(ResidualBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.activation1 = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride= 1, padding= 2)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.activation2 = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride= 1, padding= 2)
    
    def forward(self,x):
        residual  = torch.clone(x)
        x = self.bn1(x)
        x = self.activation1(x)
        x = self.conv1(x)
        x = self.conv2(self.activation2(self.bn2(x)))
        residual = residual.unsqueeze(0)
        residual = nn.functional.interpolate(residual, size = [x.shape[1], x.shape[2], x.shape[3]])
        residual = residual.squeeze(0)
        x += residual
        return x
    

class CNNModel(nn.Module):

    def __init__(self):
        super(CNNModel, self).__init__()
        self.feature = nn.Sequential()
        self.feature.add_module('f_conv1', nn.Conv2d(in_channels = 5, out_channels = 32, kernel_size= 3, stride= 1, padding= 2))
        self.feature.add_module('f_resblock1', ResidualBlock(32,32,5))
        self.feature.add_module('f_resblock2', ResidualBlock(32,64,5))
        self.feature.add_module('f_resblock3', ResidualBlock(64,128,5))
        self.feature.add_module('f_adaptiveavgpool', nn.AdaptiveAvgPool2d((15,15)))

        self.class_classifier = nn.Sequential()
        self.class_classifier.add_module('c_fc1', nn.Linear(15 * 15 * 128, 1024))
        self.class_classifier.add_module('c_bn1', nn.BatchNorm1d(1024))
        self.class_classifier.add_module('c_relu1', nn.ReLU(True))
        self.class_classifier.add_module('c_drop1', nn.Dropout(0.2))
        self.class_classifier.add_module('c_fc2', nn.Linear(1024, 512))
        self.class_classifier.add_module('c_bn2', nn.BatchNorm1d(512))
        self.class_classifier.add_module('c_relu2', nn.ReLU(True))
        self.class_classifier.add_module('c_fc3', nn.Linear(512, 2))
        self.class_classifier.add_module('c_softmax', nn.Softmax(dim = 1))

        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(15 * 15 * 128, 1024))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(1024))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(1024, 2))
        self.domain_classifier.add_module('d_softmax', nn.Softmax(dim = 1))
    

    def forward(self, input_data, alpha):
        feature = self.feature(input_data)
        feature = feature.view(-1, 15 * 15 * 128)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)
        return class_output, domain_output


In [130]:
class Dataset(Dataset):
    def __init__(self, data, info):
        #data loading
        self.x = data
        self.y = info
        self.n_samples = data.shape[0]


    def __getitem__(self,index):
        t1 = self.x[index]
        t2 = self.y[index]
        t1 = torch.tensor(t1)
        t1 = t1.permute((2,0,1))
        t2 = torch.tensor(t2)
        return (t1,t2)
    
    def __len__(self):
        return self.n_samples
    
data = scipy.io.loadmat('/home/desktop/Desktop/22104412_Docs/EEG-COGMusic/Baseline/datasets/s01_datasets_Zscore_clipped.mat')

x_c=data['coh']
x_p = data['pli']
x_d = data['psd']
labels_skf = data['labels_kfold']
labels = data['valence']

In [131]:
x_n = np.zeros((40,75,32,32,5))
for trial in range(x_n.shape[0]):
    for sample in range(x_n.shape[1]):
        x_n[trial,sample,:,:,0] = x_c[trial,sample,:,:,0]+np.transpose(x_p[trial,sample,:,:,0])
        x_n[trial,sample,:,:,1] = x_c[trial,sample,:,:,1]+np.transpose(x_p[trial,sample,:,:,1])
        x_n[trial,sample,:,:,2] = x_c[trial,sample,:,:,2]+np.transpose(x_p[trial,sample,:,:,2])
        x_n[trial,sample,:,:,3] = x_c[trial,sample,:,:,3]+np.transpose(x_p[trial,sample,:,:,3])
        x_n[trial,sample,:,:,4] = x_c[trial,sample,:,:,4]+np.transpose(x_p[trial,sample,:,:,4])
    print(f'Completed trial {trial}')

for trial in range(x_n.shape[0]):
    for sample in range(x_n.shape[1]):
        for i in range(32):
            x_n[trial,sample,i,i,:] = x_d[trial,sample,i,:]
    print(f'Completed trial {trial}')

Completed trial 0
Completed trial 1
Completed trial 2
Completed trial 3
Completed trial 4
Completed trial 5
Completed trial 6
Completed trial 7
Completed trial 8
Completed trial 9
Completed trial 10
Completed trial 11
Completed trial 12
Completed trial 13
Completed trial 14
Completed trial 15
Completed trial 16
Completed trial 17
Completed trial 18
Completed trial 19
Completed trial 20
Completed trial 21
Completed trial 22
Completed trial 23
Completed trial 24
Completed trial 25
Completed trial 26
Completed trial 27
Completed trial 28
Completed trial 29
Completed trial 30
Completed trial 31
Completed trial 32
Completed trial 33
Completed trial 34
Completed trial 35
Completed trial 36
Completed trial 37
Completed trial 38
Completed trial 39
Completed trial 0
Completed trial 1
Completed trial 2
Completed trial 3
Completed trial 4
Completed trial 5
Completed trial 6
Completed trial 7
Completed trial 8
Completed trial 9
Completed trial 10
Completed trial 11
Completed trial 12
Completed tri

In [132]:
v = labels_skf[:,0]
indices = np.where((v>5.5)|(v<4.5))[0]
indices.shape

(38,)

In [133]:
dt = x_n[indices]
labels = labels[indices]
trainLoss={}
valLoss = {}
trainAcc = {}
valAcc = {}

In [134]:
l_skf = np.zeros([40,1])
l_skf[np.where(labels_skf[:,0]>5)[0]] = 1
l_skf = l_skf[indices]

In [135]:
dt.shape,labels.shape,l_skf.shape

((38, 75, 32, 32, 5), (38, 75, 2), (38, 1))

In [136]:
train_index[-4:].shape

(4,)

In [137]:
kf = skf(n_splits = 10)
log_pred_dict = {}
for k,(train_index,test_index) in enumerate(kf.split(dt, l_skf)):
    print(f'Fold {k+1} running')
    dataTr, dataV, dataTest = np.concatenate(dt[train_index[0:-4]],0),np.concatenate(dt[train_index[-4:]],0), np.concatenate(dt[test_index],0)
    labelsTr, labelsV, labelsTest = np.concatenate(labels[train_index[0:-4]],0),np.concatenate(labels[train_index[-4:]],0), np.concatenate(labels[test_index],0)
    tAV = []
    vAV = []
    y_train = np.argmax(labelsTr, axis=1)
    class_counts = np.bincount(y_train)
    num_classes = len(class_counts)
    total_samples = len(y_train)

    class_weights = []
    for count in class_counts:
        weight = 1 / (count / total_samples)
        class_weights.append(weight)
    
    weights_class = torch.tensor(np.array(class_weights))
    weights_class = weights_class.cuda()

    weights_domain = torch.tensor(np.array([1.05,5.5]))
    ## parameters
    bs = 100
    image_size = 32
    n_epoch = 50

    ## load data
    dataset_source = Dataset(dataTr, labelsTr)
    dataloader_source = DataLoader(dataset = dataset_source, batch_size = bs, shuffle = True)
 
    dataset_target = Dataset(dataV, labelsV)
    dataloader_target = DataLoader(dataset = dataset_target, batch_size = 10, shuffle = True)
    
    testDS = Dataset(dataTest, labelsTest)
    testDL = DataLoader(dataset = testDS, batch_size = 100, shuffle = True )

    ###### model
    my_net = CNNModel()
     # Define your weights as a tensor
    optimizer = optim.SGD(my_net.parameters(), lr=0.0001, momentum= 0.9)
    loss_class = nn.CrossEntropyLoss(weight=weights_class)
    loss_domain = nn.CrossEntropyLoss(weight=weights_domain)

    cuda = True
    cudnn.benchmark = True

    if cuda:
        my_net = my_net.cuda()
        loss_class = loss_class.cuda()
        loss_domain = loss_domain.cuda()

    for p in my_net.parameters():
        p.requires_grad = True

    #### training    
    for epoch in range(n_epoch):

        len_dataloader = min(len(dataloader_source), len(dataloader_target))
        data_source_iter = iter(dataloader_source)
        data_target_iter = iter(dataloader_target)

        i = 0
        while i < len_dataloader:

            p = float(i + epoch * len_dataloader) / n_epoch / len_dataloader
            alpha = 2. / (1. + np.exp(-10 * p)) - 1

            # training model using source data
            data_source = next(data_source_iter)
            source_img, source_label = data_source

            my_net.zero_grad()
            batch_size = len(source_label)

            domain_label = torch.zeros(batch_size,2)
            domain_label[:,0] = 1
            domain_label = domain_label.long()

            if cuda:
                source_img = source_img.cuda()
                source_label = source_label.cuda()
                domain_label = domain_label.cuda()

            class_output, domain_output = my_net(input_data=source_img.float(), alpha=alpha)
            _, class_label = torch.max(source_label,dim=1)
            err_s_label = loss_class(class_output, source_label.float())
            err_s_domain = loss_domain(domain_output, domain_label.float())

            # training model using target data
            data_target = next(data_target_iter)
            target_img, _ = data_target

            batch_size = len(target_img)

            domain_label = torch.zeros(batch_size,2)
            domain_label[:,1] = 1
            domain_label = domain_label.long()

            if cuda:
                target_img = target_img.cuda()
                domain_label = domain_label.cuda()

            _, domain_output = my_net(input_data=target_img.float(), alpha=alpha)
            err_t_domain = loss_domain(domain_output, domain_label.float())
            err = err_t_domain + err_s_domain + err_s_label
            err.backward()
            optimizer.step()

            i += 1

            # print ('epoch: %d, [iter: %d / all %d], err_s_label: %f, err_s_domain: %f, err_t_domain: %f' \
            #     % (epoch, i, len_dataloader, err_s_label.cpu().data.numpy(),
            #         err_s_domain.cpu().data.numpy(), err_t_domain.cpu().data.numpy()))

        #tAV.append(test(dataloader_source, epoch, my_net, 'source'))
        #vAV.append(test(dataloader_target, epoch, my_net, 'target'))
        vAV.append(test(testDL, epoch, my_net,'test'))
    #trainAcc[str(k)] = tAV
    valAcc[str(k)] = vAV
    print(f'Fold {k} Completed')

Fold 1 running
Epoch 0 for test dataset is 50.666666666666664
Epoch 1 for test dataset is 49.333333333333336
Epoch 2 for test dataset is 50.333333333333336
Epoch 3 for test dataset is 50.333333333333336
Epoch 4 for test dataset is 46.333333333333336
Epoch 5 for test dataset is 54.0
Epoch 6 for test dataset is 49.333333333333336
Epoch 7 for test dataset is 42.666666666666664
Epoch 8 for test dataset is 49.0
Epoch 9 for test dataset is 42.0
Epoch 10 for test dataset is 45.0
Epoch 11 for test dataset is 45.333333333333336
Epoch 12 for test dataset is 46.0
Epoch 13 for test dataset is 47.666666666666664
Epoch 14 for test dataset is 43.666666666666664
Epoch 15 for test dataset is 48.666666666666664
Epoch 16 for test dataset is 44.333333333333336
Epoch 17 for test dataset is 45.666666666666664
Epoch 18 for test dataset is 44.0
Epoch 19 for test dataset is 48.333333333333336
Epoch 20 for test dataset is 47.666666666666664
Epoch 21 for test dataset is 47.666666666666664
Epoch 22 for test datas

In [141]:
vAV = np.array(list(valAcc.values()))

In [142]:
VA = np.max(vAV,1)
print(VA)

[54.         56.66666667 59.66666667 54.66666667 60.66666667 60.66666667
 54.33333333 57.66666667 71.11111111 64.44444444]


In [143]:
np.mean(VA)

59.38888888888889